# RAG 智能问答系统

---

通过RAG可以给大模型灌输一些客户自己的资料，然后让大模型能够尝试理解客户的自然提问，结合灌输的资料给出更合理、更精确的答案。接下来，我们将会整理一套美团的常见问题说明文档，整理给大模型，让大模型能够更好的理解美团的重要业务流程。

> 说明：通常RAG更适合用于处理客户自己的，不太适合对互联网公开的资料。这个例子中，我们用互联网上的资料做为案例，仅仅只是用来介绍RAG的用法。

## 1. RAG 基础流程

虽然大模型也依然能够给出一个答案，但是，很显然，他并没有专门"学习"美团的业务知识，自然就无法针对美团的常见问题说明，给出比较理想的答案。

那么，如何让大模型能够"学习"美团的业务知识呢？典型的方法有两种：
 
1. 一种方法是微调，就是让大模型回炉进行一次学习，你可以理解为让一个已经毕业的博士，回炉重新学习美团的业务知识，通常像通义千问、deepseeek这样的大模型，经过复杂，庞大的学习过程，已经具备了非常好的理解能力和知识储备，这些称为预训练模型，pretraining。而微调就是在这些预训练模型的基础上，再投入少量的资源，让大模型继续学习。微调的过程，就是让大模型学习到我们自己收集的数据，让大模型"学习"到我们自己收集的数据，从而达到更好的理解能力。这种方式针对性比较强，对问题的立即感也能更精确。但是，随之而来的问题是成本还是比较高，而且实现的难度比较大。如果没有足够的功底，甚至可能让大模型产生倒退。

2. 另一种方法就是RAG，Retrival-Augmented Generation 检索增强生成。这种方法就是在询问大模型问题时，先检索出跟问题可能相关的参考信息，然后再把问题跟参考信息一起输入给大模型，让大模型结合参考信息，给出更好的答案。你可以理解为我们去问一个专家询问相关业务问题时，带上业务手册，这样专家就不用提前学习额外的知识，只需要结合业务手册，就能快速给出针对性的帮助。这种方式成本比较低，也更适合处理那些涉及到大量外部数据的特定问题。因此，RAG也是目前企业用得做多的一种方法。

> 通常，RAG和模型微调也是可以混合使用的，RAG是给大模型额外四肢，而模型微调是给大模型补脑子，两者相辅相成。

RAG 的基础工作流程通常分为两个阶段：Indexing 索引阶段和 Retrieval 检索阶段。

1. Indexing 索引阶段：

这一阶段主要是要对相关文档进行预处理，形成知识库，便于后续检索。通常需要将各种形式的文档转化成为Document，然后将Document拆分成小段的Segments，然后将这些Segments进行Embedding向量化处理，并将结果保存到向量数据库当中，这样，后续的检索工作就可以直接使用向量数据库进行检索了。

```mermaid
 graph LR
     A[文档] --> B[Document]
     B --> C[Segments]
     C --> D[Embedding]
     D --> E[向量数据库]
```

2. Retrieval 检索阶段：

这一阶段主要是当用户提出一个问题时，可以到向量数据库中检索出跟用户的问题比较关联的Segment，把这些segment和用户的问题一起整理成完整的prompt，再发送给大模型，然后由大模型对信息金正整合，再给用户返回正确答案。

> 例如可以定制这样的一个prompt模板：

> prompt_template = """你是一个问答机器人，你的任务是根据下述给定的已知信息回答用户问题。已知信息：{context} # {context} 就是检索出来的文档 用户问：{question} # {question} 就是用户的问题 如果已知信息不包含用户问题的答案，或者已知信息不足以回答用户的问题，请直接回复"我无法回答您的问题"。请不要输出已知信息中不包含的信息或答案。请用中文回答用户问题。"""


## Indexing 索引阶段

### 1. 加载并解析文档

langchain中提供了非常多Document Loader工具，可以从PDF、HTML、MarkDown、JSON、CSV等各种格式的文档中加载数据，甚至还实现了非常多的扩展工具，可以从网页上加载数据。使用这些工具，可以很方便的加载文档，例如：

In [3]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./example_data/question_answer.txt", encoding="utf-8")

documents = loader.load()

documents

[Document(metadata={'source': './example_data/question_answer.txt'}, page_content='1. 问题: 美团外卖配送费是如何计算的?\n答案: 配送费根据配送距离、天气情况、高峰时段等因素综合计算,一般在3-8元不等。\n\n2. 问题: 如何申请美团外卖退款?\n答案: 可以在订单详情页点击"申请退款",选择退款原因并上传相关凭证,等待商家审核。紧急情况可以联系美团客服处理。\n\n3. 问题: 美团外卖订单超时了怎么办?\n答案: 如果超过预计送达时间,可以在APP查看骑手位置或致电骑手询问。如果严重延误可以申请赔付或退款。\n\n4. 问题: 美团外卖可以修改收货地址吗?\n答案: 下单后如果订单还未被商家接单,可以点击订单详情中的"修改地址"进行修改。如果已被接单则无法修改。\n\n5. 问题: 如何成为美团外卖商家?\n答案: 需要准备营业执照、食品经营许可证等资质,在美团商家版APP上提交入驻申请,通过审核后即可上线经营。\n\n6. 问题: 美团外卖骑手迟到了可以投诉吗?\n答案: 可以。在订单详情页面点击"投诉建议",选择"配送问题"进行反馈,平台会进行处理并给予回复。\n\n7. 问题: 美团外卖红包怎么使用?\n答案: 下单时在支付页面选择"使用红包",系统会自动显示可用的红包,选择合适的红包即可抵扣相应金额。\n\n8. 问题: 美团外卖订单送错了怎么办?\n答案: 及时联系骑手说明情况,如果骑手已离开,可以联系商家或美团客服处理,一般可以重新配送或退款。\n\n9. 问题: 怎样加入美团外卖骑手团队?\n答案: 需要年满18周岁,有电动车和智能手机,在美团骑手APP上注册并提交身份证、健康证等资料,通过培训考核后即可上岗。\n\n10. 问题: 美团外卖商家的评分是如何计算的?\n答案: 商家评分由用户评价、配送时效、投诉率等多个维度综合计算得出,满分5星。评分会影响商家的排名展示和接单量。\n\n')]

在LangChain中，TextLoader是BaseLoader的一个实现类。除了TextLoader外，还有非常多的实现类。例如，如果你的知识库文件比较多，你可以尝试使用DirectoryLoader，它会自动遍历的加载文件夹中的所有文件。

In [4]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

directLoader = DirectoryLoader("./example_data", glob="**/*.txt", loader_cls=TextLoader, show_progress=True)

directLoader.load()

  0%|          | 0/1 [00:00<?, ?it/s]Error loading file example_data\question_answer.txt


RuntimeError: Error loading example_data\question_answer.txt

### 2. 切分文档

接下来，需要将Documents中的文件切分成一个个比较独立的Segments。一个Segments表示一个比较独立的知识片段。例如，对于我们这个示例，可以把一个问答就当成一个segment。实现时，就是按照“\n\n”两个换行符进行切分。 

In [4]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./example_data/question_answer.txt", encoding="utf-8")

documents = loader.load()

# 切分文档
text_splitters = CharacterTextSplitter(chunk_size=500, chunk_overlap=0, separator="\n\n", keep_separator=True)

segments = text_splitters.split_documents(documents)

print(len(segments))

for segment in segments:
    print(segment.page_content)
    print('-----------------')

2
1. 问题: 美团外卖配送费是如何计算的?
答案: 配送费根据配送距离、天气情况、高峰时段等因素综合计算,一般在3-8元不等。

2. 问题: 如何申请美团外卖退款?
答案: 可以在订单详情页点击"申请退款",选择退款原因并上传相关凭证,等待商家审核。紧急情况可以联系美团客服处理。

3. 问题: 美团外卖订单超时了怎么办?
答案: 如果超过预计送达时间,可以在APP查看骑手位置或致电骑手询问。如果严重延误可以申请赔付或退款。

4. 问题: 美团外卖可以修改收货地址吗?
答案: 下单后如果订单还未被商家接单,可以点击订单详情中的"修改地址"进行修改。如果已被接单则无法修改。

5. 问题: 如何成为美团外卖商家?
答案: 需要准备营业执照、食品经营许可证等资质,在美团商家版APP上提交入驻申请,通过审核后即可上线经营。

6. 问题: 美团外卖骑手迟到了可以投诉吗?
答案: 可以。在订单详情页面点击"投诉建议",选择"配送问题"进行反馈,平台会进行处理并给予回复。
-----------------
7. 问题: 美团外卖红包怎么使用?
答案: 下单时在支付页面选择"使用红包",系统会自动显示可用的红包,选择合适的红包即可抵扣相应金额。

8. 问题: 美团外卖订单送错了怎么办?
答案: 及时联系骑手说明情况,如果骑手已离开,可以联系商家或美团客服处理,一般可以重新配送或退款。

9. 问题: 怎样加入美团外卖骑手团队?
答案: 需要年满18周岁,有电动车和智能手机,在美团骑手APP上注册并提交身份证、健康证等资料,通过培训考核后即可上岗。

10. 问题: 美团外卖商家的评分是如何计算的?
答案: 商家评分由用户评价、配送时效、投诉率等多个维度综合计算得出,满分5星。评分会影响商家的排名展示和接单量。
-----------------


测试时，你会发现，CharacterTextSplitter组件切分出来的结果可能和预期结果相比没有那么精确。文档中有 10 个问答条目，而切分的结果只有 2 个。这是因为在做RAG文档切分时，通常我们并不需要像传统数据库那样严格的切分文档。最终基于大模型强大的理解能力，即便不是很合理的拆分，也能得到比较好的效果。当然，如果想要按照每个单独的问答进行严格的切分，也不是没有办法。我们也可以自行切分内容。 

In [6]:
import re
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("./example_data/question_answer.txt", encoding="utf-8")

documents = loader.load()

# 切分文档
text_splitters = CharacterTextSplitter(chunk_size=500, chunk_overlap=0, separator="\n\n", keep_separator=True)

# 自行拆分文档
texts = re.split(r"\n\n", documents[0].page_content)

# 将文档片段转换为 documents

segment_documents = text_splitters.create_documents(texts)

print(len(segment_documents))

for segment in segment_documents:
    print(segment.page_content)
    print('-----------------')

10
1. 问题: 美团外卖配送费是如何计算的?
答案: 配送费根据配送距离、天气情况、高峰时段等因素综合计算,一般在3-8元不等。
-----------------
2. 问题: 如何申请美团外卖退款?
答案: 可以在订单详情页点击"申请退款",选择退款原因并上传相关凭证,等待商家审核。紧急情况可以联系美团客服处理。
-----------------
3. 问题: 美团外卖订单超时了怎么办?
答案: 如果超过预计送达时间,可以在APP查看骑手位置或致电骑手询问。如果严重延误可以申请赔付或退款。
-----------------
4. 问题: 美团外卖可以修改收货地址吗?
答案: 下单后如果订单还未被商家接单,可以点击订单详情中的"修改地址"进行修改。如果已被接单则无法修改。
-----------------
5. 问题: 如何成为美团外卖商家?
答案: 需要准备营业执照、食品经营许可证等资质,在美团商家版APP上提交入驻申请,通过审核后即可上线经营。
-----------------
6. 问题: 美团外卖骑手迟到了可以投诉吗?
答案: 可以。在订单详情页面点击"投诉建议",选择"配送问题"进行反馈,平台会进行处理并给予回复。
-----------------
7. 问题: 美团外卖红包怎么使用?
答案: 下单时在支付页面选择"使用红包",系统会自动显示可用的红包,选择合适的红包即可抵扣相应金额。
-----------------
8. 问题: 美团外卖订单送错了怎么办?
答案: 及时联系骑手说明情况,如果骑手已离开,可以联系商家或美团客服处理,一般可以重新配送或退款。
-----------------
9. 问题: 怎样加入美团外卖骑手团队?
答案: 需要年满18周岁,有电动车和智能手机,在美团骑手APP上注册并提交身份证、健康证等资料,通过培训考核后即可上岗。
-----------------
10. 问题: 美团外卖商家的评分是如何计算的?
答案: 商家评分由用户评价、配送时效、投诉率等多个维度综合计算得出,满分5星。评分会影响商家的排名展示和接单量。
-----------------


### 3. 文本向量化 + 保存向量数据库

切分出我们需要的知识条目后，就可以对文本进行向量化，并将这些向量化的结果保存到向量数据库当中。这里还是以之前介绍过的Redis作为示例 

In [13]:
import os
import re
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_redis import RedisConfig, RedisVectorStore
from langchain_text_splitters import CharacterTextSplitter

load_dotenv()

loader = TextLoader("./example_data/question_answer.txt", encoding="utf-8")

documents = loader.load()

# 切分文档
text_splitters = CharacterTextSplitter(chunk_size=500, chunk_overlap=0, separator="\n\n", keep_separator=True)

# 自行拆分文档
texts = re.split(r"\n\n", documents[0].page_content)

# 将文档片段转换为 documents

segment_documents = text_splitters.create_documents(texts)

# 构建向量化模型
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# 使用 Redis 构建向量数据库
redis_url = "redis://localhost:6379"

redis_config = RedisConfig(
    index_name="meituan_index",
    redis_url=redis_url,
    distance_metric="COSINE"
)

vector_store = RedisVectorStore(embedding_model, config=redis_config)

# 将文档保存到向量数据库中
vector_store.add_documents(segment_documents)

['meituan_index:01JSF79D5BAGH4H05MQJEFEMNC',
 'meituan_index:01JSF79D5BJNYD46PA3K0PJ9H1',
 'meituan_index:01JSF79D5BVK3ZVF9H326MB8BN',
 'meituan_index:01JSF79D5BNF17XT050ZS1194T',
 'meituan_index:01JSF79D5B2GBW7NBFE1JNB982',
 'meituan_index:01JSF79D5B8S3QKMVFVHKYGY3G',
 'meituan_index:01JSF79D5BN9FD8TY8J4DX5DQA',
 'meituan_index:01JSF79D5BCRBYW2NA93ABTH0D',
 'meituan_index:01JSF79D5BFQRPKFVW33MMC1MB',
 'meituan_index:01JSF79D5B71XBZ49TAER644ZJ']

到这里就完成了 RAG 的第一个阶段，Indexing 索引阶段。也就是说知识库内容处理完成。接下来就可以进入第二个阶段 Retrival，对客户问题做检索增强了。最后，把这些代码整合到一起，总结一下 RAG 建立本地消息索引的过程。

## Retrival 检索增强阶段

在这个阶段，主要是要围绕客户提出的问题做一些补充和优化。在接收到用户的一个问题后，我们需要先到向量数据库中去检索一下跟用户提出的问题相关的知识条目。这样未来就可以把用户的问题和本地知识库中相关的知识条目一起发给大模型，让大模型综合考虑之后，给出一个理想的答案。 

### 4. 检索相关信息

比如，用户询问“订单超时了怎么办”这样的问题，我们就需要先到Redis中检索一下跟这个问题相关的知识条目有哪些。 

In [14]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_redis import RedisConfig, RedisVectorStore

load_dotenv()

# 构建向量化模型
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# 使用 Redis 向量数据库
redis_url = "redis://localhost:6379"

redis_config = RedisConfig(
    index_name="meituan_index",
    redis_url=redis_url,
    distance_metric="COSINE"
)

vector_store = RedisVectorStore(embedding_model, config=redis_config)

retriever = vector_store.as_retriever();

relative_segments = retriever.invoke('订单超时了怎么办', k=5)

relative_segments

01:04:23 redisvl.index.index INFO   Index already exists, not overwriting.


[Document(metadata={}, page_content='3. 问题: 美团外卖订单超时了怎么办?\n答案: 如果超过预计送达时间,可以在APP查看骑手位置或致电骑手询问。如果严重延误可以申请赔付或退款。'),
 Document(metadata={}, page_content='8. 问题: 美团外卖订单送错了怎么办?\n答案: 及时联系骑手说明情况,如果骑手已离开,可以联系商家或美团客服处理,一般可以重新配送或退款。'),
 Document(metadata={}, page_content='2. 问题: 如何申请美团外卖退款?\n答案: 可以在订单详情页点击"申请退款",选择退款原因并上传相关凭证,等待商家审核。紧急情况可以联系美团客服处理。'),
 Document(metadata={}, page_content='6. 问题: 美团外卖骑手迟到了可以投诉吗?\n答案: 可以。在订单详情页面点击"投诉建议",选择"配送问题"进行反馈,平台会进行处理并给予回复。'),
 Document(metadata={}, page_content='4. 问题: 美团外卖可以修改收货地址吗?\n答案: 下单后如果订单还未被商家接单,可以点击订单详情中的"修改地址"进行修改。如果已被接单则无法修改。')]

### 5. 构建 Prompt 提示词

查询出跟用户问题相关的“知识”后，就需要将用户的问题和相关的“知识”整合到一起，才能发给大模型。

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_redis import RedisConfig, RedisVectorStore

load_dotenv()

# 构建向量化模型
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# 使用 Redis 向量数据库
redis_url = "redis://localhost:6379"

redis_config = RedisConfig(
    index_name="meituan_index",
    redis_url=redis_url,
    distance_metric="COSINE"
)

vector_store = RedisVectorStore(embedding_model, config=redis_config)

retriever = vector_store.as_retriever();

relative_segments = retriever.invoke('订单超时了怎么办', k=5)

prompt_template = ChatPromptTemplate.from_messages([
    ("human", """你是一个答疑机器人，你的任务是根据下面已知信息回答用户的问题。
     已知信息：{context}
     用户问题：{question}
     如果已知信息不包含用户的答案，或者已知信息不足以回答用户的问题，请直接回复“我无法回答您的问题”。
     """)
])

text = []

for segment in relative_segments:
    text.append(segment.page_content)

prompt = prompt_template.invoke({ "context": text, "question": "订单超时了怎么办" })

prompt

01:12:50 redisvl.index.index INFO   Index already exists, not overwriting.


ChatPromptValue(messages=[HumanMessage(content='你是一个答疑机器人，你的任务是根据下面已知信息回答用户的问题。\n     已知信息：[\'3. 问题: 美团外卖订单超时了怎么办?\\n答案: 如果超过预计送达时间,可以在APP查看骑手位置或致电骑手询问。如果严重延误可以申请赔付或退款。\', \'8. 问题: 美团外卖订单送错了怎么办?\\n答案: 及时联系骑手说明情况,如果骑手已离开,可以联系商家或美团客服处理,一般可以重新配送或退款。\', \'2. 问题: 如何申请美团外卖退款?\\n答案: 可以在订单详情页点击"申请退款",选择退款原因并上传相关凭证,等待商家审核。紧急情况可以联系美团客服处理。\', \'6. 问题: 美团外卖骑手迟到了可以投诉吗?\\n答案: 可以。在订单详情页面点击"投诉建议",选择"配送问题"进行反馈,平台会进行处理并给予回复。\', \'4. 问题: 美团外卖可以修改收货地址吗?\\n答案: 下单后如果订单还未被商家接单,可以点击订单详情中的"修改地址"进行修改。如果已被接单则无法修改。\']\n     用户问题：订单超时了怎么办\n     如果已知信息不包含用户的答案，或者已知信息不足以回答用户的问题，请直接回复“我无法回答您的问题”。\n     ', additional_kwargs={}, response_metadata={})])

### 6. 调用大模型

将包含背景知识和用户问题的 Prompt 发送给大模型

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_redis import RedisConfig, RedisVectorStore

load_dotenv()

# 构建向量化模型
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# 使用 Redis 向量数据库
redis_url = "redis://localhost:6379"

redis_config = RedisConfig(
    index_name="meituan_index",
    redis_url=redis_url,
    distance_metric="COSINE"
)

vector_store = RedisVectorStore(embedding_model, config=redis_config)

retriever = vector_store.as_retriever();

relative_segments = retriever.invoke('订单超时了怎么办', k=5)

prompt_template = ChatPromptTemplate.from_messages([
    ("human", """你是一个答疑机器人，你的任务是根据下面已知信息回答用户的问题。
     已知信息：{context}
     用户问题：{question}
     如果已知信息不包含用户的答案，或者已知信息不足以回答用户的问题，请直接回复“我无法回答您的问题”。
     """)
])

text = []

for segment in relative_segments:
    text.append(segment.page_content)

prompt = prompt_template.invoke({ "context": text, "question": "订单超时了怎么办" })

llm = ChatOpenAI(model="gpt-4o-mini")

response = llm.invoke(prompt)

response.content

01:19:34 redisvl.index.index INFO   Index already exists, not overwriting.


'如果订单超时了，可以在APP查看骑手位置或致电骑手询问。如果严重延误，可以申请赔付或退款。'

## 总结

这里我们快速的实现了一个简单的RAG应用，但是，对RAG的思考其实远不止于此。RAG应用的核心是在向AI大模型询问问题时，尽量提供更高质量的参考信息，但是，最终AI大模型给出的回答靠不靠谱，我们却无法保证。颇有点尽人事，听天命的感觉。但是，作为应用开发者，我们是要对最终的答案负责。那么，要如何提高RAG应用的最终质量呢？这就需要我们对RAG的流程重新进行深度思考。这里，不妨再来回顾一下RAG的基础流程，我们重点思考一下几个问题：

1. 如何验证RAG应用的质量？

样本测试，相似度检查

RAGAS

2. 如何提升RAG应用的质量？ 

数据质量高，结构完整

文本拆分，拆分章节，设置假设性的问题

图片 OCR 识别，生成文字；表格生成 Markdown 格式

提升提示词的质量

用户问题拆分/转写，拆成多个问题，多个 chain 提问；或者拆分多个步骤（可以让大模型做）

检索的结果，相似度高的问题放前面，重新排序，Rerank

